Importacion de librerias
se usara Numpy y se ajustara la reprodccion del entrenamiento ademas se importaran librerias para el manejo de lo que es la arquitectura de la red neuronal recurrente

In [21]:
#Freddy Alejandro Florez Bohorquez
#Inteligencia Artificial
#Enlace https://colab.research.google.com/drive/1Xz3Q3jvb50Jn7ETsRqUV66Fmi2OdN3Ak?usp=sharing

import numpy as np                                 #Numpy para el manejo de arrays
np.random.seed(5)

from tensorflow.keras.layers import Input, Dense, SimpleRNN  #para la capa de entrada, la capa dense y en especial la cell state
from tensorflow.keras.models import Model                    #Despues importamos el modelo y el modelo y el optimizador SGD que nos permitira el entreno
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical    #para las salidas al modelo en el formato adecuado durante el entrenamiento y prediccion
from keras import backend as K

En esta parte preparamos el dataset y los guarda en diccionarios

In [22]:
# 1. LECTURA DEL SET DE DATOS
# ===========================================================
nombres = open('/content/drive/MyDrive/RNN/nombres_dinosaurios.txt','r').read()
nombres = nombres.lower()

# Crear diccionario (listado de caracteres que no se repiten)
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (tam_datos, tam_alfabeto))

# Conversión de caracteres a índices y viceversa
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))} #equivalencia correspondiente entre el caracter y el indice correspondiente one- hot
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))} #generar indices no caracteres


En total hay 19909 caracteres, y el diccionario tiene un tamaño de 27 caracteres.


In [23]:
#asi se evidencia que cada caracter corresponde a un instante de tiempo de la red recurrente
print(car_a_ind)
print(ind_a_car)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


En esta parte se compilara el modelo ademas se le adicionan todo el entorno de la red para ello comenzando con las entradas la celda de recurrencia de la red y las salidas

In [24]:
# 2. MODELO
# ===========================================================
n_a = 25    # Número de unidades en la capa oculta
entrada  = Input(shape=(None,tam_alfabeto))   #el None permite nombres de longitud variable
a0 = Input(shape=(n_a,))                      # el estado oculto (at-1 en el diagrama anterior), que será precisamente un vector con 25 elementos

celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True) #activacion y usaremos el reurn_state para el estado oculto actualizado
capa_salida = Dense(tam_alfabeto, activation='softmax')     #Esta capa tiene 27 neuronas de salida el tamaño del alfabeto y se usa la funcion activacion softmax

salida = []                                         #en esta lista se va a ir almacenando la entrega recurrende en la celda
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida.append(capa_salida(hs))
modelo = Model([entrada,a0],salida)
#modelo.summary()

opt = SGD(learning_rate=0.0005)                     #se crea un optimizador del gradiente y lo añadimos al modelo
modelo.compile(optimizer=opt, loss='categorical_crossentropy')

Para el entrenamiento generamos inicialmente una lista con los nombres de cada dinosaurio y los mezclamos aleatoriamente, usando np.random.shuffle luego Después definimos una función que tome uno a uno cada ejemplo de entrenamiento y que genere tres vectores, que serán las entradas al modelo

In [25]:
# 3. EJEMPLOS DE ENTRENAMIENTO
# ===========================================================

# Crear lista con ejemplos de entrenamiento y mezclarla aleatoriamente
with open("/content/drive/MyDrive/RNN/nombres_dinosaurios.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

# Crear ejemplos de entrenamiento usando un generador
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y


La entrada X , correspondiente a un nombre de dinosaurio tomado aleatoriamente del set de entrenamiento. Cada carácter será representado primero como un número entero (usando el diccionario creado al inicio del código) y luego representado en formato one-hot. Entrenaremos el modelo para que el primer carácter presentado sea siempre un vector de ceros

La salida Y, que será simplemente el resultado de tomar la entrada y desplazarla un carácter a la derecha. De nuevo, cada carácter es representado con un índice correspondiente y luego convertido al formato one-hot

In [33]:
# 4. ENTRENAMIENTO
# ===========================================================
BATCH_SIZE = 80			# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 1000			# Número de iteraciones

for j in range(NITS):
    historia = modelo.fit(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0) #Entrenamos el modelo, las epocas 

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    if j%1000 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')


Iteración: 0, Error: 2.719550



Finalmente definimos la activación inicial que será simplemente una matriz de ceros, y que se irá actualizando a medida que se presentan uno a uno los caracteres al modelo durante el entrenamiento.
Para entrenar esta Red Recurrente definimos un total de 10000 iteraciones, en cada una de las cuales presentaremos 80 ejemplos de entrenamiento.

El entrenamiento lo haremos con el método fit_generator, y como parámetros de entrada tendremos la función creada anteriormente (train_generator) que permite crear un ejemplo de entrenamiento, así como el número de ejemplos a usar durante cada iteración (es decir 80)

In [34]:
# 5. GENERACIÓN DE NOMBRES USANDO EL MODELO ENTRENADO
# ===========================================================
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          y = capa_salida(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 50):
            nombre_generado += '\n'

    print(nombre_generado)

# Generar 10 ejemplos de nombres generados por el modelo ya entrenado
for i in range(10):
    generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)

aund

ra

saus

oqus

igaur

oreciprirurusorusgiosrus

auruhvunrus

dlpuroraxonrurrurlntout

whorhurus

haus

